<a href="https://colab.research.google.com/github/seunghee0518/python_colab/blob/main/tensorflow_car_evaluation_%EC%9E%90%EB%A3%8C%EB%A1%9C_%EB%B6%84%EB%A5%98_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#install& import library

In [ ]:
#library install
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn

In [ ]:
#(must!) upgrade
!pip install --upgrade pandas --use-feature=2020-resolver
!pip install --upgrade matplotlib --use-feature=2020-resolver
!pip install --upgrade seaborn --use-feature=2020-resolver

In [ ]:
#tensorflow library
!pip install -q tensorflow-gpu==2.0.0-rc1

In [ ]:
#import library
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")

# load data

In [ ]:
#data link : https://github.com/gilbutITbook/080263

In [ ]:
cols = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety', 'output']
car_url = 'https://raw.githubusercontent.com/gilbutITbook/080263/master/chap2/data/car_evaluation.csv'
cars = pd.read_csv(car_url, names=cols, header = None)
cars

In [ ]:
#pie-chart
plot_size = plt.rcParams["figure.figsize"]
plot_size[0] = 8
plot_size[1] = 6
plt.rcParams["figure.figsize"] = plo
t_size
cars.output.value_counts().plot(kind='pie', autopct = '%0.05f%%', colors = ['lightblue', 'lightgreen', 'orange', 'pink'] , explode = (0.05, 0.05, 0.05, 0.05))

* output(차 상태)
  * unacc : 허용 불가능 상태
  * acc : 허용 가능한 수준
  * good : 양호
  * very good : 매우 좋은 상태
* 70%가 허용 불가능

#preprocess